In [ ]:
%load_ext autoreload
%autoreload 2
import json 
import pysftp
import os
import pandas_gbq
import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
import logging

#Configure loggging
logging.basicConfig(filename='BigQuery.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New Big Query Logging Instance')

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact, import the SFTP password from the same file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-db919282054b.json'
with open('powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    sftp_pass = j['sftp_password']
    
# ----------------------------------------------------------
#Need to make this portion to where it assesses all files in the dir recursively. 
def main(SFTP_folder_name, sftp_pool):

    SFTP_folder_name  = initial_schema_check(SFTP_folder_name)
    print(SFTP_folder_name)

    instance = Create(
                project_id='powerschool-420113',
                location = 'us-south1',
                bucket=f'{SFTP_folder_name}bucket-iotaschools-1',
                local_dir = f'SFTP_folders\\{SFTP_folder_name}',
                db = SFTP_folder_name,
                append_or_replace='replace',
                )

    SFTP_conn(SFTP_folder_name, sftp_pool) #Replicate SFTP folder onto local dir
    instance.process()#Replicate local dir into Buckets, pass dir files into Bucket & finally into Big Query tables


# Initialize the connection pool outside of the main process and pass in as an arg
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None 
sftp_pool = SFTPConnectionPool(
    host="sftp.iotaschools.org",
    username="iota.sftp",
    password=sftp_pass, 
    cnopts=cnopts
)


main("powerschool", sftp_pool)
main("EIS", sftp_pool)

# Close all connections when done
sftp_pool.close_all_connections()

#When running two at one time it does not get all files from the SFTP initially. 
#Therefore connection open & closes. 

#Took 4 minutes to get the powerschool folder downloaded. 
#Less than a minute to send files to Bucket, and Big Query

#EIS 5 min mark
#Issue with downloading the EIS_prior_schools.csv
#Could be a way with how I am opening and closing the connection to where it hangs on it. 
#Went for over an hour. 

#Going to either need to download all files at one time, or re-frame the conection. 
#Look into connection pooling

#The file immediately ends up on there, but that function simply hangs. 

#Next one started at 4:40
#Immediately got downloaded in the first 12 seconds. 
#Finished in 6 mins

#7 mins PS started, immediately downloaded files. Then hangs
#Finished in 13 minutes
#Never closed the connection

#Ran in 16 mins